In [261]:
from matplotlib import pyplot as plt
import numpy as np
import random
import time
import math

In [273]:
class Individual:
    def __init__(self, functions, lb, ub):
        self.functions = functions
        self.lb = lb
        self.ub = ub
        self.code = np.random.random(len(lb)) * (ub - lb) + lb
        self.fvalue = np.empty(len(functions))
        self.evaluate()
        
        self.rank = math.inf
        self.crowding_distance = float('inf')

    def evaluate(self):
        for i in range(len(self.functions)):
            self.fvalue[i] = self.functions[i](self.code)

    def __str__(self):
        return self.code

In [228]:
# Function to find Pareto front

def find_pareto_front(population):
    pop_size = len(population)
    fNum = len(population[0].functions)
    pareto_front = []
    pareto_front_ids = []
    
    for i in range(pop_size):
        is_pareto_optimal = True

        for j in range(pop_size):
            if i != j:
                is_dominated = True
                for k in range(fNum):
                    is_dominated &= population[i].fvalue[k] > population[j].fvalue[k]
                if is_dominated:
                    is_pareto_optimal = False
                    break
        if is_pareto_optimal:
            pareto_front.append(population[i])
            pareto_front_ids.append(i)

    return pareto_front_ids, pareto_front

In [263]:
# Non-dominated sorting algorithm

def non_dominated_sort(population, max_fronts):
    removable_population = np.array(population.copy())
    pareto_fronts = []
    rank = 1

    for i in range(max_fronts):
        if len(removable_population) == 0:
            break
        pareto_front_ids, pareto_front = find_pareto_front(removable_population)
        pareto_fronts.append(pareto_front)
        for individual in pareto_front:
            individual.rank = rank
        removable_population = np.delete(removable_population, pareto_front_ids)
        rank += 1
    #return pareto_fronts

In [277]:
def calculate_crowding_distance(front):
    fNum = len(front[0].functions)
    crowding_distances = np.zeros(len(front))
    
    for k in range(fNum):
        front.sort(key = lambda x: x.fvalue[k])

        crowding_distances[0] = float('inf')
        crowding_distances[-1] = float('inf')

        min_fvalue = front[0].fvalue[k]
        max_fvalue = front[-1].fvalue[k]

        if max_fvalue == min_fvalue:
            continue

        for i in range(1, len(front) - 1):
            crowding_distances[i] += (front[i+1].fvalue[k] - front[i-1].fvalue[k]) / (max_fvalue - min_fvalue)

    for i in range(len(front)):
        front[i].crowding_distance = crowding_distances[i]
    #return crowding_distances

In [278]:
# Tournament selection, picking a random member of the tournaments Pareto front

def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)

    pareto_front = find_pareto_front(chosen)

    return random.choice(pareto_front)

In [279]:
# Whole arithmetic recombination

def crossover(parent1, parent2, child1, child2, alpha = 0.5):
    for i in range(len(child1.code)):
        child1.code[i] = parent1.code[i] * alpha + parent2.code[i] * (1 - alpha)
        child2.code[i] = parent2.code[i] * alpha + parent1.code[i] * (1 - alpha)

In [280]:
# Uniform mutation

def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if np.random.random() < mutation_prob:
            individual.code[i] = np.random.random() * (individual.ub[i] - individual.lb[i]) + individual.lb[i]

In [281]:
def nsga(functions, lb, ub, pop_size, num_generations, max_fronts, tournament_size, elitism_size, crossover_alpha, mutation_prob, showGraph = True):
    start_time = time.time()
    
    # Initializing population
    population = [Individual(functions, lb, ub) for _ in range(pop_size)]
    new_population = population.copy()

    # Sorting individuals by dominance
    pareto_fronts = non_dominated_sort(population, max_fronts)

    crowding_distances = []
    for front in pareto_fronts:
        calculate_crowding_distance(front)

    for individual in population:
        print(individual.code, individual.rank, individual.crowding_distance)

In [286]:
# Schaffer function N.2
# -5 <= x <= 10

def f1(x):
    if x[0] <= 1:
        return -x[0]
    elif x[0] <= 3:
        return x[0] - 2
    elif x[0] <= 4:
        return 4 - x[0]
    else:
        return x[0] - 4
        
def f2(x):
    return (x[0] - 5)**2

lower_bounds = np.array([-5])
upper_bounds = np.array([10])

nsga(functions=[f1,f2], lb=lower_bounds, ub=upper_bounds, pop_size=100, num_generations=100, max_fronts = 100, tournament_size=5, elitism_size=50, crossover_alpha=0.3, mutation_prob=0.05)

[0.55244413] 4 inf
[-2.36804325] 21 inf
[-3.91975522] 29 inf
[7.97511608] 28 inf
[-0.79902286] 13 inf
[-0.27887811] 9 inf
[8.36833012] 33 inf
[9.84996764] 44 inf
[0.61621142] 2 inf
[7.18400389] 21 inf
[1.97863796] 1 0.8284052864529869
[4.0455106] 1 0.7585540343990539
[3.94143954] 2 2.0
[5.93661162] 9 inf
[-2.29025674] 20 inf
[6.46922381] 16 inf
[7.3183177] 22 inf
[7.13260546] 20 inf
[8.3638202] 31 inf
[-1.00786531] 16 inf
[8.63515945] 36 inf
[8.3427175] 30 inf
[-3.70859507] 28 inf
[-0.49480493] 10 inf
[9.9492035] 46 inf
[9.41072559] 40 inf
[1.13453789] 1 inf
[2.36005322] 5 1.6417176883535003
[5.4785982] 3 inf
[7.08082659] 19 inf
[8.06561874] 29 inf
[-3.02991199] 24 inf
[0.12690204] 5 inf
[6.35452986] 13 inf
[5.5337416] 4 inf
[2.6813837] 8 1.4903267407475222
[6.89431466] 18 inf
[1.61470434] 1 0.43582658819882514
[7.64268753] 24 inf
[9.62257741] 41 inf
[3.32564475] 7 0.48617428097773707
[3.67638242] 4 2.0
[6.19940561] 11 inf
[3.13642401] 9 2.0
[5.14736357] 1 inf
[-3.41082265] 27 inf
[-1.

In [284]:
# Schaffer function N.1
# -A <= X <= A, where A is between 10 and 10e5, higher A's increasing the difficulty of the problem

def f1(x):
    return x[0]**2

def f2(x):
    return (x[0]-2)**2

A = 10
lower_bounds = np.array([-A])
upper_bounds = np.array([A])

nsga(functions=[f1,f2], lb=lower_bounds, ub=upper_bounds, pop_size=1000, num_generations=100, max_fronts = 1000, tournament_size=5, elitism_size=50, crossover_alpha=0.3, mutation_prob=0.05)

[-0.76869516] 47 inf
[-0.65888804] 33 inf
[-8.61107259] 440 inf
[0.81333601] 1 0.0512906798657398
[5.28158677] 154 inf
[-6.76927601] 358 inf
[-5.38061413] 286 inf
[-8.83332746] 447 inf
[4.15511256] 90 inf
[-4.6737223] 242 inf
[3.89995994] 82 inf
[-4.57571214] 235 inf
[-6.51225087] 348 inf
[-1.38139074] 75 inf
[-4.3764986] 226 inf
[-3.47095418] 171 inf
[-8.40465336] 431 inf
[7.17854014] 256 inf
[-7.64673419] 402 inf
[-8.37588474] 429 inf
[3.40852423] 58 inf
[0.04758888] 1 0.04981155105126997
[-1.6911802] 87 inf
[1.99221865] 1 inf
[-2.14565017] 110 inf
[9.08123313] 356 inf
[-4.46191203] 229 inf
[-9.82195546] 501 inf
[7.7625423] 281 inf
[-8.88071791] 453 inf
[-8.75186005] 446 inf
[-9.68755499] 495 inf
[-7.13251464] 382 inf
[1.18314143] 1 0.024765498820002015
[-5.91426218] 314 inf
[9.55706633] 383 inf
[7.89203963] 287 inf
[5.75730109] 182 inf
[-1.92820289] 103 inf
[-1.2692395] 72 inf
[-5.6412837] 298 inf
[-0.03531332] 4 inf
[0.24339397] 1 0.08189030642502648
[-5.94777488] 320 inf
[-9.50824